<a href="https://colab.research.google.com/github/AdarshShah/final-year-project/blob/master/project_sepsis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [0]:
import time
import pandas as pd
import psycopg2
import getpass
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Declare Global Variables

In [0]:
user = 'postgres'
host = 'localhost'
dbname = 'mimic'
schema = 'mimiciii'
password = 'postgres'
icd9_code = '99592'
minMaxScaler = dict()
itemMeans = dict()

# Create Database Connection

In [0]:
conn = psycopg2.connect(user=user,password=password,dbname=dbname,host=host)
cur = conn.cursor()
cur.execute('SET search_path to {}'.format(schema))

# Load Patient data




> Data from admissions suffering from severe sepsis

In [0]:
query = """SELECT DISTINCT * FROM mimiciii.admissions i 
INNER JOIN mimiciii.diagnoses_icd USING(hadm_id)
INNER JOIN mimiciii.d_icd_diagnoses USING(icd9_code)
WHERE icd9_code LIKE '99592';
"""
admissions = pd.read_sql_query(sql=query,con=conn)
admissions['sepsis'] = 1

query = """SELECT DISTINCT * FROM mimiciii.admissions i 
INNER JOIN mimiciii.diagnoses_icd USING(hadm_id)
INNER JOIN mimiciii.d_icd_diagnoses USING(icd9_code)
WHERE icd9_code NOT LIKE '99592' LIMIT 4000;
"""
temp = pd.read_sql_query(sql=query,con=conn)
temp['sepsis'] = 0

admissions = admissions.append(temp)



> List Items from d_labitems and d_items

In [0]:
#d_items = ['Platelets','C Reactive Protein (CRP)','pH (Art)','HCO3','TCO2 (cap)']
#d_labitems = ['Leukocytes','Urea Nitrogen','Creatinine','Glucose','Calcium, Total','Sodium','Potassium','Bilirubin','Albumin','Lactate','pCO2','pO2','Hemoglobin']

#Append itemid below !!!!
d_item_ce = ['0','828','3789','6256','791','3750','1525','220615','811','3744','3745','1529','226537','837','3803','1536','4948','3066','227444','3835','3837','812','3736']  
d_item_cv = ['0','30006','44711','44441']
d_item_mv = ['0','225170']
d_labitem = ['51486','51006','50912','50809','50931','51478','50813','50820']

In [0]:
def get_pandas(items, table):
  query = "SELECT DISTINCT * FROM mimiciii."+table+" WHERE itemid IN " + str(tuple(items))
  items = pd.read_sql_query(sql=query,con=conn)
  return items

d_item_ce = get_pandas(d_item_ce,"d_items")
d_item_cv = get_pandas(d_item_cv,"d_items")
d_item_mv = get_pandas(d_item_mv,"d_items")
d_labitem = get_pandas(d_labitem,"d_labitems")

# Fetch items from mimic-iii

> CHARTEVENTS

In [0]:
query = "SELECT * FROM mimiciii.chartevents WHERE itemid IN " + str(tuple(d_item_ce['itemid'])) + " AND hadm_id IN " + str(tuple(admissions['hadm_id'])) + " ORDER BY hadm_id,charttime,itemid;"
chartevents = pd.read_sql_query(sql=query,con=conn)


In [0]:
itemids = chartevents['itemid'].drop_duplicates()
temp = chartevents.pivot(index='row_id',columns='itemid',values='valuenum')
temp.fillna(value=temp.mean(skipna=True),inplace=True)
temp.replace()
for item in itemids:
  arr = np.reshape(np.array(temp[item]),(-1,1))
  minMaxScaler[item] = MinMaxScaler().fit(arr)

In [0]:
temp = dict(temp.mean())
for key in temp.keys():
  itemMeans[key] = temp[key]

In [307]:
chartevents.describe()

,row_id,subject_id,hadm_id,icustay_id,itemid,cgid,valuenum,warning,error
count,4.187140e+05,418714.000000,418714.000000,418180.000000,418714.000000,418714.000000,417160.000000,55838.000000,55838.000000
mean,1.018948e+08,21693.081619,149650.986881,251298.341984,30587.669846,16198.020756,101.430435,0.583456,0.002651
std,5.540498e+07,19398.070517,28713.531852,28650.827482,75101.349121,1972.237414,93.739899,0.492990,0.051415
min,1.565500e+04,21.000000,100074.000000,200030.000000,791.000000,14023.000000,-70.000000,0.000000,0.000000
25%,5.614203e+07,8623.000000,125102.000000,227178.000000,811.000000,15331.000000,2.700000,0.000000,0.000000
50%,1.009089e+08,18105.000000,149230.000000,252147.000000,837.000000,15331.000000,116.000000,1.000000,0.000000
75%,1.514004e+08,27574.000000,175201.000000,275623.000000,1529.000000,15331.000000,142.000000,1.000000,0.000000
max,1.931220e+08,99991.000000,199943.000000,299998.000000,227444.000000,21570.000000,3555.000000,1.000000,1.000000


> INPUTEVENTS_CV



In [0]:
query = "SELECT * FROM mimiciii.inputevents_cv WHERE itemid IN " + str(tuple(d_item_cv['itemid'])) + " AND hadm_id IN " + str(tuple(admissions['hadm_id'])) + " ORDER BY hadm_id,charttime,itemid;"
inputevents_cv = pd.read_sql_query(sql=query,con=conn)

In [0]:
itemids = inputevents_cv['itemid'].drop_duplicates()
temp = inputevents_cv.pivot(index='row_id',columns='itemid',values='amount')
temp.fillna(value=temp.mean(skipna=True),inplace=True)
for item in itemids:
  arr = np.reshape(np.array(temp[item]),(-1,1))
  minMaxScaler[item] = MinMaxScaler().fit(arr)

In [0]:
temp = dict(temp.mean())
for key in temp.keys():
  itemMeans[key] = temp[key]

In [311]:
inputevents_cv.describe()

,row_id,subject_id,hadm_id,icustay_id,itemid,amount,cgid,orderid,linkorderid,newbottle
count,2.526000e+03,2526.000000,2526.000000,2514.000000,2526.000000,2356.000000,2526.000000,2.526000e+03,2.526000e+03,310.0
mean,4.699851e+06,16084.254157,147882.490499,251681.871122,30272.076801,207.120020,18089.475455,6.017716e+06,6.057863e+06,1.0
std,1.733265e+06,8802.873451,30184.285114,29871.128362,1954.146186,121.367148,2363.535887,3.425607e+06,3.419688e+06,0.0
min,4.457295e+06,117.000000,100253.000000,200102.000000,30006.000000,0.000000,14023.000000,4.580000e+03,9.591000e+03,1.0
25%,4.460475e+06,9575.000000,119862.000000,225703.000000,30006.000000,197.000000,15900.000000,3.042577e+06,3.042551e+06,1.0
50%,4.463914e+06,15812.000000,150850.000000,255660.000000,30006.000000,232.000000,17701.000000,6.058852e+06,6.254552e+06,1.0
75%,4.466991e+06,22844.000000,179019.000000,276363.000000,30006.000000,269.000000,20297.000000,8.912993e+06,9.190013e+06,1.0
max,1.745546e+07,32658.000000,199716.000000,299994.000000,44711.000000,1099.000000,21570.000000,1.195363e+07,1.190499e+07,1.0


> INPUTEVENTS_MV

In [0]:
query = "SELECT * FROM mimiciii.inputevents_mv WHERE itemid IN ('225170') AND hadm_id IN " + str(tuple(admissions['hadm_id'])) + " ORDER BY hadm_id,starttime,itemid;"
inputevents_mv = pd.read_sql_query(sql=query,con=conn)

In [0]:
itemids = inputevents_mv['itemid'].drop_duplicates()
temp = inputevents_mv.pivot(index='row_id',columns='itemid',values='amount')
temp.fillna(value=temp.mean(skipna=True),inplace=True)
for item in itemids:
  arr = np.reshape(np.array(temp[item]),(-1,1))
  minMaxScaler[item] = MinMaxScaler().fit(arr)

In [0]:
temp = dict(temp.mean())
for key in temp.keys():
  itemMeans[key] = temp[key]

In [315]:
inputevents_mv.describe()

,row_id,subject_id,hadm_id,icustay_id,itemid,amount,rate,cgid,orderid,linkorderid,patientweight,totalamount,isopenbag,continueinnextdept,cancelreason,originalamount,originalrate
count,1.561000e+03,1561.000000,1561.000000,1560.000000,1561.0,1561.000000,1561.000000,1561.00000,1.561000e+03,1.561000e+03,1561.000000,1561.000000,1561.000000,1561.0,1561.000000,1561.000000,1561.000000
mean,1.743785e+06,62290.476618,148569.034593,250600.185256,225170.0,247.385181,340.450911,17913.09417,4.993048e+06,4.993893e+06,85.569122,250.942345,0.000641,0.0,0.127482,252.388843,340.311749
std,1.083833e+06,22758.434495,28044.280488,30952.150530,0.0,107.673826,267.668683,2250.58988,2.880190e+06,2.872215e+06,21.716758,74.339057,0.025310,0.0,0.432369,102.465548,267.535428
min,2.940000e+02,124.000000,100442.000000,200454.000000,225170.0,-20.099999,1.900000,14023.00000,1.289000e+04,1.289000e+04,1.000000,130.000000,0.000000,0.0,0.000000,70.000000,1.900000
25%,7.255180e+05,50065.000000,123675.000000,218620.000000,225170.0,198.999996,200.999994,15830.00000,2.545321e+06,2.549322e+06,70.000000,199.000000,0.000000,0.0,0.000000,199.000000,201.000000
50%,1.712049e+06,60762.000000,142534.000000,248745.000000,225170.0,230.999994,260.000010,17661.00000,4.992425e+06,4.999181e+06,85.000000,233.000000,0.000000,0.0,0.000000,233.000000,260.000000
75%,2.800644e+06,80030.000000,175115.000000,281049.000000,225170.0,281.000004,393.999996,20030.00000,7.504859e+06,7.483251e+06,96.700000,282.000000,0.000000,0.0,0.000000,282.000000,394.000000
max,3.605911e+06,99865.000000,199821.000000,299533.000000,225170.0,3010.000200,6020.000400,21492.00000,9.992465e+06,9.992465e+06,170.500000,757.000000,1.000000,0.0,2.000000,3010.000000,6020.000000


> LABEVENTS

In [0]:
query = "SELECT * FROM mimiciii.labevents WHERE itemid IN " + str(tuple(d_labitem['itemid'])) + " AND hadm_id IN " + str(tuple(admissions['hadm_id'])) + " ORDER BY hadm_id,charttime,itemid;"
labevents = pd.read_sql_query(sql=query,con=conn)

In [317]:
itemids = labevents['itemid'].drop_duplicates()
temp = labevents.pivot(index='row_id',columns='itemid',values='valuenum')
temp.fillna(value=temp.mean(skipna=True),inplace=True)
for item in itemids:
  arr = np.reshape(np.array(temp[item]),(-1,1))
  minMaxScaler[item] = MinMaxScaler().fit(arr)

/home/adarsh/.local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:355: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/home/adarsh/.local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:356: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [0]:
temp = dict(temp.mean())
for key in temp.keys():
  itemMeans[key] = temp[key]

In [319]:
labevents.describe()

,row_id,subject_id,hadm_id,itemid,valuenum
count,6.244410e+05,624441.000000,624441.000000,624441.000000,598899.000000
mean,1.521366e+07,36207.164467,149334.049881,50929.465394,46.213613
std,8.294357e+06,29101.123171,28569.363457,135.933690,68.616224
min,1.021200e+04,21.000000,100074.000000,50809.000000,0.000000
25%,7.843078e+06,12581.000000,124911.000000,50820.000000,3.000000
50%,1.630106e+07,26313.000000,148758.000000,50912.000000,7.490000
75%,2.246933e+07,58702.000000,174734.000000,51006.000000,86.000000
max,2.790723e+07,99991.000000,199943.000000,51486.000000,3565.000000


# Generate Tensors

In [0]:
hadm_id_ce = chartevents['hadm_id'].drop_duplicates()
hadm_id_cv = inputevents_cv['hadm_id'].drop_duplicates()
hadm_id_mv = inputevents_mv['hadm_id'].drop_duplicates()
hadm_id_lb = labevents['hadm_id'].drop_duplicates()
hadm_id = hadm_id_ce.append(hadm_id_cv.append(hadm_id_mv.append(hadm_id_lb))).drop_duplicates()
hadm_id.describe()

In [0]:
sepsis_dataset = []
for hadm in hadm_id[0:10]:
  data = pd.DataFrame(data=itemMeans,index={0})

  temp = chartevents[chartevents['hadm_id'] == hadm]
  temp = temp.pivot(index='row_id',columns='itemid',values='valuenum')
  data = data.append(temp,ignore_index=False)
  
  temp = inputevents_cv[inputevents_cv['hadm_id'] == hadm]
  temp = temp.pivot(index='row_id',columns='itemid',values='amount')
  data = data.append(temp,ignore_index=False)
  
  temp = inputevents_mv[inputevents_mv['hadm_id'] == hadm]
  temp = temp.pivot(index='row_id',columns='itemid',values='amount')
  data = data.append(temp,ignore_index=False)

  temp = labevents[labevents['hadm_id'] == hadm]
  temp = temp.pivot(index='row_id',columns='itemid',values='valuenum')
  data = data.append(temp,ignore_index=False)

  data.fillna(value=itemMeans,inplace=True)
  data['sepsis'] = int(admissions[admissions['hadm_id']==hadm]['sepsis'].all())
  for key in minMaxScaler.keys():
    arr = np.reshape(np.array(data[key]),(-1,1))
    data[key] = minMaxScaler[key].transform(arr)
  sepsis_dataset.insert(len(sepsis_dataset),data)